In [1]:
from bloqade import save, load 
from bloqade.atom_arrangement import Chain 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
if not os.path.isdir("data"): 
    os.mkdir("data")

In [ ]:
initial_geometry = Chain(2, lattice_spacing="distance")
program_waveforms = initial_geometry.rydberg.rabi.amplitude.uniform.piecewise_linear(
    durations=["ramp_time", "run_time", "ramp_time"],
    values=[0.0, "rabi_ampl", "rabi_ampl", 0.0],
)
program_assigned_vars = program_waveforms.assign(
    ramp_time=0.06, rabi_ampl=15, distance=8.5
)
batch = program_assigned_vars.batch_assign(run_time=0.05 * np.arange(31))

In [ ]:
emu_filename = os.path.join(
    os.path.abspath(""), "data", "nonequilibrium-dynamics-blockade-emulation.json"
)
if not os.path.isfile(emu_filename):
    emu_batch = batch.braket.local_emulator().run(10000)
    save(emu_batch, emu_filename)

In [ ]:
filename = os.path.join(os.path.abspath(""), "data", "nonequilibrium-dynamics-blockade-job.json")

if not os.path.isfile(filename):
    hardware_batch = batch.parallelize(24).braket.aquila().run_async(shots=100)
    save(hardware_batch, filename)

In [ ]:
def rydberg_state_probabilities(shot_counts):
    probabilities_dict = {"0": [], "1": [], "2": []}

    # iterate over each of the task results
    for task_result in shot_counts:
        # get total number of shots
        total_shots = sum(task_result.values())
        # get probability of each state
        probabilities_dict["0"].append(task_result.get("11", 0) / total_shots)
        probabilities_dict["1"].append(
            (task_result.get("10", 0) + task_result.get("01", 0)) / total_shots
        )
        probabilities_dict["2"].append(task_result.get("00", 0) / total_shots)

    return probabilities_dict

In [ ]:
emu_batch = load(emu_filename)
emu_report = emu_batch.report()
emu_counts = emu_report.counts()

#hardware_batch = load(filename)
# hardware_batch.fetch() # uncomment to fetch results from Braket
# save(filename, hardware_batch

In [ ]:
emu_report = emu_batch.report()
hardware_report = hardware_batch.report()


emu_rydberg_state_probabilities = rydberg_state_probabilities(emu_report.counts())
#hw_rydberg_state_probabilities = rydberg_state_probabilities(hardware_report.counts())

In [ ]:
figure, axs = plt.subplots(1, 3, figsize=(12, 6), sharey=True)

emu_run_times = emu_report.list_param("run_time")
#hardware_run_times = hardware_report.list_param("run_time")

emu_colors = ["#55DE79", "#EDFF1A", "#C2477F"]  # Green, Yellow, Red

emu_lines = []
#hw_lines = []
for ax, rydberg_state, color in zip(axs, ["0", "1", "2"], emu_colors):
    (hw_line,) = ax.plot(
        emu_run_times,
        hw_rydberg_state_probabilities[rydberg_state],
        label=rydberg_state + "-Rydberg QPU",
        color=color,
    )
    (emu_line,) = ax.plot(
        hardware_run_times,
        emu_rydberg_state_probabilities[rydberg_state],
        color="#878787",
    )
    emu_line.set_label("Emulator")

    emu_lines.append(emu_line)
    hw_lines.append(hw_line)

    ax.set_xlabel("time ($\mu s$)")
    ax.set_ylabel("Probability")

ax.legend(handles=[*hw_lines, emu_lines[-1]])

plt.show()